In [5]:
import numpy as np

class LunarLanderEnv:
    """
    Lunar Lander environment simulator (2D) implementing the provided mathematical model.
    Only implements initializer and a `step(action)` method as requested.

    State vector:
        s = [X_com, Y_com, v_x, v_y, theta, omega, left_contact, right_contact]

    Actions:
        0: no thrust
        1: left thruster (produces a rightward force in body-frame) -> sets F_left (in formulas)
        2: main engine (upward in body-frame)
        3: right thruster (produces a leftward force in body-frame) -> sets F_right

    Notes:
        - Integration method: semi-implicit Euler (stable default).
        - Leg contact: checks leg-tip world y <= ground_y(leg_x). When contact occurs, contact flag sets to 1.
        - Episode termination: when Y_com <= 0 OR either leg contact occurs OR max_steps reached.
        - Terminal reward: +100 for safe landing per spec; -100 otherwise.
    """
    def __init__(self,
                 m=1.0,              # mass
                 I=1.0,              # moment of inertia
                 g=9.81,             # gravity (downwards)
                 dt=0.02,            # simulation timestep
                 main_force=20.0,    # thrust magnitude for main engine (tunable)
                 side_force=5.0,     # thrust magnitude for side thrusters (tunable)
                 integration="semi-implicit",  # "kinematic" or "semi-implicit"
                 reward_coeffs=None, # dict of reward coefficients (uses recommended defaults if None)
                 max_steps=1000,
                 initial_state=None, # provide 8-element initial state or let env sample default
                 rng_seed=None
                 ):
        # physics params
        self.m = float(m)
        self.I = float(I)
        self.g = float(g)
        self.dt = float(dt)
        self.main_force = float(main_force)
        self.side_force = float(side_force)
        assert integration in ("kinematic", "semi-implicit")
        self.integration = integration

        # reward coefficients (defaults from spec) -- added c_dx and c_dv for differential rewards
        if reward_coeffs is None:
            reward_coeffs = {
                # c_x and c_v left in dict for compatibility but not used anymore for differential-only mode
                'c_x': 0.0,
                'c_v': 0.0,
                'c_dx': 0.1,      # differential position coeff (new)
                'c_dv': 0.1,      # differential velocity coeff (new)
                'c_theta': 2.0,
                'c_l': 10.0,
                'c_r': 10.0,
                'c_s': 0.03,
                'c_m': 0.3
            }
        else:
            # ensure new keys exist if the user supplied a custom dict
            reward_coeffs = dict(reward_coeffs)  # shallow copy
            reward_coeffs.setdefault('c_dx', 0.1)
            reward_coeffs.setdefault('c_dv', 0.1)
            reward_coeffs.setdefault('c_x', 0.0)
            reward_coeffs.setdefault('c_v', 0.0)

        self.rc = reward_coeffs

        # step counting and termination limits
        self.max_steps = int(max_steps)
        self.step_count = 0

        # RNG
        self.rng = np.random.RandomState(rng_seed)

        # initial state
        if initial_state is not None:
            s = np.array(initial_state, dtype=float)
            if s.shape[0] != 8:
                raise ValueError("initial_state must be length 8")
            self.state = s.copy()
        else:
            # default starting pose: above pad near center, small random v and slight angle perturbation
            X0 = self.rng.uniform(-2.0, 2.0)
            Y0 = self.rng.uniform(10.0, 15.0)
            vx0 = self.rng.normal(0.0, 0.5)
            vy0 = self.rng.normal(0.0, 0.5)
            theta0 = np.pi/2.0 + self.rng.normal(0.0, 0.1)  # upright +/- small
            omega0 = self.rng.normal(0.0, 0.1)
            l0 = 0.0
            r0 = 0.0
            self.state = np.array([X0, Y0, vx0, vy0, theta0, omega0, l0, r0], dtype=float)

        # previous-step differential info (initialized at reset)
        # prev_d = |X_{t-1} - x_pad|, prev_v = speed at t-1
        self.prev_d = None
        self.prev_v = None

        # termination tracking
        self.done = False
        self.truncated = False

    # -------------------------
    # Ground height definition
    # -------------------------
    def ground_height(self, x):
        """
        Piecewise ground function Y_ground(X) as per spec.
        Uses the discrete values in the problem statement; for intervals we follow ranges.
        """
        x = float(x)
        if -16.0 <= x <= -9.0:
            return -4.0
        if np.isclose(x, -8.0):
            return -3.0
        if np.isclose(x, -7.0):
            return -2.0
        if np.isclose(x, -6.0):
            return -1.0
        if -5.0 <= x <= 7.0:
            return 0.0
        if 8.0 <= x <= 9.0:
            return 1.0
        if 10.0 <= x <= 11.0:
            return 2.0
        if np.isclose(x, 12.0):
            return 3.0
        if np.isclose(x, 13.0):
            return 2.0
        if np.isclose(x, 14.0):
            return 1.0
        if np.isclose(x, 15.0):
            return 0.0
        if np.isclose(x, 16.0):
            return 0.0
        if x < -16.0:
            return -4.0
        if x > 16.0:
            return 0.0
        return 0.0

    # -------------------------
    # Body point -> world coords
    # -------------------------
    def body_to_world(self, X_com, Y_com, theta, x_loc, y_loc):
        """
        Rotation mapping (from doc):
        x_w = X_com + sinθ * x_loc + cosθ * y_loc
        y_w = Y_com - cosθ * x_loc + sinθ * y_loc
        """
        s = np.sin(theta)
        c = np.cos(theta)
        xw = X_com + s * x_loc + c * y_loc
        yw = Y_com - c * x_loc + s * y_loc
        return xw, yw

    def leg_tips(self, state):
        """
        Returns world coords of right and left leg tips based on the state.
        According to spec:
          Right leg tip in body-frame: (+2, -2)
          Left leg tip in body-frame: (-2, -2)
        """
        X, Y, _, _, theta, _, _, _ = state
        xr, yr = self.body_to_world(X, Y, theta, 2.0, -2.0)
        xl, yl = self.body_to_world(X, Y, theta, -2.0, -2.0)
        return (xr, yr), (xl, yl)

    # -------------------------
    # Step function
    # -------------------------
    def step(self, action):
        """
        Apply action (0..3), advance the simulator one dt and return:
            next_state (8-d numpy array),
            reward (float),
            terminated (bool),
            truncated (bool),
            info (dict)
        """
        if self.done:
            return self.state.copy(), 0.0, True, self.truncated, {'reason': 'already_done'}

        if action not in (0, 1, 2, 3):
            raise ValueError("action must be in {0,1,2,3}")

        # Unpack previous state (at time t)
        X, Y, vx, vy, theta, omega, l_flag, r_flag = self.state.copy()

        # Determine thruster magnitudes
        F_main = 0.0
        F_left = 0.0
        F_right = 0.0

        if action == 1:
            F_left = self.side_force
        elif action == 3:
            F_right = self.side_force
        elif action == 2:
            F_main = self.main_force

        # Forces in world frame
        Fx = F_main * np.cos(theta) + (F_left - F_right) * np.sin(theta)
        Fy = F_main * np.sin(theta) + (-F_left + F_right) * np.cos(theta) - self.m * self.g

        # Linear accelerations
        ax = Fx / self.m
        ay = Fy / self.m

        # Torque and angular acceleration
        tau = (F_right - F_left)
        alpha = tau / self.I

        # Integration step
        dt = self.dt
        if self.integration == "kinematic":
            vx_new = vx + ax * dt
            vy_new = vy + ay * dt
            X_new = X + vx * dt + 0.5 * ax * dt * dt
            Y_new = Y + vy * dt + 0.5 * ay * dt * dt

            omega_new = omega + alpha * dt
            theta_new = theta + omega * dt + 0.5 * alpha * dt * dt
        else:
            vx_new = vx + ax * dt
            vy_new = vy + ay * dt
            X_new = X + vx_new * dt
            Y_new = Y + vy_new * dt

            omega_new = omega + alpha * dt
            theta_new = theta + omega_new * dt

        # compute leg-tip coordinates after update
        (xr, yr), (xl, yl) = self.leg_tips(np.array([X_new, Y_new, vx_new, vy_new, theta_new, omega_new, l_flag, r_flag]))

        # detect contacts
        ground_y_r = self.ground_height(xr)
        ground_y_l = self.ground_height(xl)

        eps = 1e-8
        r_contact = 1.0 if (yr <= ground_y_r + eps) else 0.0
        l_contact = 1.0 if (yl <= ground_y_l + eps) else 0.0

        # update contact flags
        l_flag_new = max(l_flag, l_contact)
        r_flag_new = max(r_flag, r_contact)

        # contact correction
        if l_contact or r_contact:
            vy_new = min(vy_new, 0.0)
            shifts = []
            if l_contact:
                if yl < ground_y_l:
                    shifts.append(ground_y_l - yl)
            if r_contact:
                if yr < ground_y_r:
                    shifts.append(ground_y_r - yr)
            if shifts:
                shift = max(shifts)
                Y_new += shift
                (xr, yr), (xl, yl) = self.leg_tips(np.array([X_new, Y_new, vx_new, vy_new, theta_new, omega_new, l_flag_new, r_flag_new]))

        # update state
        next_state = np.array([X_new, Y_new, vx_new, vy_new, theta_new, omega_new, l_flag_new, r_flag_new], dtype=float)
        self.state = next_state.copy()
        self.step_count += 1

        # -------------------------
        # Rewards computation (DIFFERENTIAL ONLY for pos & vel)
        # -------------------------
        c = self.rc
        x_pad = 0.0

        # compute current distance and speed
        d = abs(X_new - x_pad)
        speed = np.sqrt(vx_new * vx_new + vy_new * vy_new)

        # initialize prev values if None (first step after reset)
        if self.prev_d is None:
            # at first step, do not apply differential reward (delta = 0)
            self.prev_d = d
        if self.prev_v is None:
            self.prev_v = speed

        # deltas
        delta_d = d - self.prev_d
        delta_v = speed - self.prev_v

        # differential rewards (positive when moving closer/slowing down)
        r_pos = - c['c_dx'] * delta_d
        r_vel = - c['c_dv'] * delta_v

        # tilt penalty remains as before
        r_theta = - c['c_theta'] * abs(theta_new - (np.pi / 2.0))

        # leg contact bonus
        r_lr = c['c_l'] * l_flag_new + c['c_r'] * r_flag_new

        # control cost
        u_side = 1.0 if (action == 1 or action == 3) else 0.0
        u_main = 1.0 if action == 2 else 0.0
        r_ctrl = - c['c_s'] * u_side - c['c_m'] * u_main

        reward_step = r_pos + r_vel + r_theta + r_lr + r_ctrl

        # update stored previous values for next step
        self.prev_d = d
        self.prev_v = speed

        # -------------------------
        # Termination check (with viewport termination)
        # -------------------------
        terminated = False
        term_reason = None

        # viewport limits (per spec/viewport)
        X_MIN, X_MAX = -16.0, 16.0
        Y_MIN, Y_MAX = -10.0, 22.0

        # 1) OUT-OF-VIEW termination (immediate, no terminal reward/penalty)
        if (X_new < X_MIN) or (X_new > X_MAX) or (Y_new < Y_MIN) or (Y_new > Y_MAX):
            terminated = True
            term_reason = 'out_of_view'
            # do not add any terminal reward/penalty — only use the step reward
            reward = reward_step
            self.done = True
        else:
            # 2) Normal termination: contact with ground or COM below ground
            if Y_new <= 0.0 or (l_flag_new == 1.0) or (r_flag_new == 1.0):
                terminated = True
                safe = False
                if (l_flag_new == 1.0) and (r_flag_new == 1.0):
                    dx = abs(X_new - x_pad)
                    if dx <= 2.0 and abs(theta_new - (np.pi / 2.0)) <= (10.0 * np.pi / 180.0):
                        safe = True
                if safe:
                    reward_term = +100.0
                    term_reason = 'safe_landing'
                else:
                    reward_term = -100.0
                    term_reason = 'crash'
                reward = reward_step + reward_term
                self.done = True
            else:
                reward = reward_step


        # truncated if exceeded max steps
        truncated = False
        if (not self.done) and (self.step_count >= self.max_steps):
            truncated = True
            self.truncated = True
            self.done = True
            term_reason = 'max_steps'

        info = {
            'step_count': self.step_count,
            'xr': float(xr), 'yr': float(yr),
            'xl': float(xl), 'yl': float(yl),
            'ground_r': float(ground_y_r), 'ground_l': float(ground_y_l),
            'F_main': float(F_main), 'F_left': float(F_left), 'F_right': float(F_right),
            'action': int(action),
            'terminated_reason': term_reason,
            # diagnostics for tuning
            'r_pos': float(r_pos), 'r_vel': float(r_vel),
            'delta_d': float(delta_d), 'delta_v': float(delta_v)
        }

        return self.state.copy(), float(reward), bool(self.done), bool(truncated), info

    def reset(self, initial_state=None, rng_seed=None):
        """
        Reset environment to either provided initial_state or to a sampled default.
        Returns the initial state.
        """
        if rng_seed is not None:
            self.rng = np.random.RandomState(rng_seed)
        if initial_state is not None:
            s = np.array(initial_state, dtype=float)
            if s.shape[0] != 8:
                raise ValueError("initial_state must be length 8")
            self.state = s.copy()
        else:
            X0 = self.rng.uniform(-2.0, 2.0)
            Y0 = self.rng.uniform(10.0, 15.0)
            vx0 = self.rng.normal(0.0, 0.5)
            vy0 = self.rng.normal(0.0, 0.5)
            theta0 = np.pi/2.0 + self.rng.normal(0.0, 0.1)
            omega0 = self.rng.normal(0.0, 0.1)
            self.state = np.array([X0, Y0, vx0, vy0, theta0, omega0, 0.0, 0.0], dtype=float)

        # initialize prev_d and prev_v based on the reset state (so first step delta=0)
        x_pad = 0.0
        X0, Y0, vx0, vy0 = float(self.state[0]), float(self.state[1]), float(self.state[2]), float(self.state[3])
        self.prev_d = abs(X0 - x_pad)
        self.prev_v = np.sqrt(vx0*vx0 + vy0*vy0)

        self.step_count = 0
        self.done = False
        self.truncated = False
        return self.state.copy()


In [6]:
import numpy as np

# -------------------------
# Example usage: NOOP policy (let it fall with good initial condition)
# -------------------------
if __name__ == "__main__":

    # Good initial condition: top center, no initial push, perfectly vertical
    X0 = 0.0            # top center horizontally
    Y0 = 20.0           # near top of viewport (high enough to fall)
    vx0 = 0.0           # no horizontal push
    vy0 = 0.0           # no vertical push
    theta0 = np.pi / 2  # perfectly vertical
    omega0 = 0.0        # no angular velocity
    l0 = 0.0
    r0 = 0.0

    init_state = [X0, Y0, vx0, vy0, theta0, omega0, l0, r0]

    env = LunarLanderEnv(
        dt=0.02,
        main_force=30.0,
        side_force=5.0,
        max_steps=2000,
        initial_state=init_state
    )

    state = env.reset(initial_state=init_state)
    print("initial state:", state)

    total_reward = 0.0

    def print_step(step, state, step_reward, total_reward):
        X, Y, vx, vy = state[0], state[1], state[2], state[3]
        Lc, Rc = int(state[6]), int(state[7])
        print(
            f"step={step:03d} | action={action} | X={X:6.3f} Y={Y:6.3f} | "
            f"vx={vx:6.3f} vy={vy:6.3f} | "
            f"r_step={step_reward:7.3f} | R={total_reward:8.3f} | "
            f"L={Lc} R={Rc}"
        )

    for t in range(1, 2001):
        action = 0   # NOOP every step

        state, r, done, truncated, info = env.step(action)
        total_reward += r

        print_step(t, state, r, total_reward)

        if done:
            reason = info.get('terminated_reason', None)
            print("\nEpisode finished at step", info['step_count'], "reason=", reason)
            if reason == 'safe_landing':
                print("Result: SAFE LANDING (+100 terminal reward applied).")
            elif reason == 'crash':
                print("Result: CRASH (-100 terminal penalty applied).")
            else:
                print("Result: Terminated (reason:", reason, ")")
            break

    if not done:
        print("\nReached max loop without termination. Final state:", state)
        print("Cumulative reward:", total_reward)


initial state: [ 0.         20.          0.          0.          1.57079633  0.
  0.          0.        ]
step=001 | action=0 | X= 0.000 Y=19.996 | vx= 0.000 vy=-0.196 | r_step= -0.020 | R=  -0.020 | L=0 R=0
step=002 | action=0 | X= 0.000 Y=19.988 | vx= 0.000 vy=-0.392 | r_step= -0.020 | R=  -0.039 | L=0 R=0
step=003 | action=0 | X= 0.000 Y=19.976 | vx= 0.000 vy=-0.589 | r_step= -0.020 | R=  -0.059 | L=0 R=0
step=004 | action=0 | X= 0.000 Y=19.961 | vx= 0.000 vy=-0.785 | r_step= -0.020 | R=  -0.078 | L=0 R=0
step=005 | action=0 | X= 0.000 Y=19.941 | vx= 0.000 vy=-0.981 | r_step= -0.020 | R=  -0.098 | L=0 R=0
step=006 | action=0 | X= 0.000 Y=19.918 | vx= 0.000 vy=-1.177 | r_step= -0.020 | R=  -0.118 | L=0 R=0
step=007 | action=0 | X= 0.000 Y=19.890 | vx= 0.000 vy=-1.373 | r_step= -0.020 | R=  -0.137 | L=0 R=0
step=008 | action=0 | X= 0.000 Y=19.859 | vx= 0.000 vy=-1.570 | r_step= -0.020 | R=  -0.157 | L=0 R=0
step=009 | action=0 | X= 0.000 Y=19.823 | vx= 0.000 vy=-1.766 | r_step= -0.020

In [7]:
import numpy as np

# -------------------------
# Crash Example (NOOP policy, crash visualization)
# -------------------------
if __name__ == "__main__":

    # Intentional crash starting condition
    X0 = 10.0            # start on the right hillside
    Y0 = 12.0            # moderate altitude
    vx0 = -2.5           # moving left fast toward slope
    vy0 = -0.5           # descending
    theta0 = np.pi/2 - 0.3   # tilted forward
    omega0 = 0.0
    l0 = 0.0
    r0 = 0.0

    init_state = [X0, Y0, vx0, vy0, theta0, omega0, l0, r0]

    env = LunarLanderEnv(
        dt=0.02,
        main_force=30.0,
        side_force=5.0,
        max_steps=500,
        initial_state=init_state
    )

    state = env.reset(initial_state=init_state)
    print("Crash test initial state:", state)

    total_reward = 0.0

    def print_step(step, action, state, step_reward, total_reward):
        X, Y, vx, vy = state[0], state[1], state[2], state[3]
        Lc, Rc = int(state[6]), int(state[7])
        print(
            f"step={step:03d} | "
            f"action={action} | "
            f"X={X:6.3f} Y={Y:6.3f} | "
            f"vx={vx:6.3f} vy={vy:6.3f} | "
            f"r={step_reward:7.3f} | "
            f"R={total_reward:8.3f} | "
            f"L={Lc} R={Rc}"
        )

    for t in range(1, 501):

        action = 0   # NOOP

        state, r, done, truncated, info = env.step(action)
        total_reward += r

        print_step(t, action, state, r, total_reward)

        if done:
            print("\nEpisode terminated at step", info["step_count"])
            print("Reason:", info["terminated_reason"])
            break

    print("Total reward:", total_reward)
    print("Final state:", state)


Crash test initial state: [10.         12.         -2.5        -0.5         1.27079633  0.
  0.          0.        ]
step=001 | action=0 | X= 9.950 Y=11.986 | vx=-2.500 vy=-0.696 | r= -0.600 | R=  -0.600 | L=0 R=0
step=002 | action=0 | X= 9.900 Y=11.968 | vx=-2.500 vy=-0.892 | r= -0.601 | R=  -1.200 | L=0 R=0
step=003 | action=0 | X= 9.850 Y=11.946 | vx=-2.500 vy=-1.089 | r= -0.602 | R=  -1.803 | L=0 R=0
step=004 | action=0 | X= 9.800 Y=11.921 | vx=-2.500 vy=-1.285 | r= -0.603 | R=  -2.406 | L=0 R=0
step=005 | action=0 | X= 9.750 Y=11.891 | vx=-2.500 vy=-1.481 | r= -0.604 | R=  -3.011 | L=0 R=0
step=006 | action=0 | X= 9.700 Y=11.858 | vx=-2.500 vy=-1.677 | r= -0.605 | R=  -3.616 | L=0 R=0
step=007 | action=0 | X= 9.650 Y=11.820 | vx=-2.500 vy=-1.873 | r= -0.606 | R=  -4.222 | L=0 R=0
step=008 | action=0 | X= 9.600 Y=11.779 | vx=-2.500 vy=-2.070 | r= -0.607 | R=  -4.830 | L=0 R=0
step=009 | action=0 | X= 9.550 Y=11.733 | vx=-2.500 vy=-2.266 | r= -0.608 | R=  -5.437 | L=0 R=0
step=010 |

In [8]:
import numpy as np

# -------------------------
# Out-of-Viewport Termination Example
# -------------------------
if __name__ == "__main__":

    # Initial condition designed to move out of viewport quickly
    X0 = 15.5          # near right boundary of viewport
    Y0 = 10.0          # safe height
    vx0 = +5.0         # strong push to the RIGHT → will exit viewport
    vy0 = 0.0
    theta0 = np.pi / 2     # upright
    omega0 = 0.0
    l0 = 0.0
    r0 = 0.0

    init_state = [X0, Y0, vx0, vy0, theta0, omega0, l0, r0]

    env = LunarLanderEnv(
        dt=0.02,
        main_force=30.0,
        side_force=5.0,
        max_steps=500,
        initial_state=init_state
    )

    state = env.reset(initial_state=init_state)
    print("Out-of-Viewport test initial state:", state)

    total_reward = 0.0

    # Logging helper (same style as your crash logger)
    def print_step(step, action, state, step_reward, total_reward):
        X, Y, vx, vy = state[0], state[1], state[2], state[3]
        Lc, Rc = int(state[6]), int(state[7])
        print(
            f"step={step:03d} | "
            f"action={action} | "
            f"X={X:6.3f} Y={Y:6.3f} | "
            f"vx={vx:6.3f} vy={vy:6.3f} | "
            f"r={step_reward:7.3f} | "
            f"R={total_reward:8.3f} | "
            f"L={Lc} R={Rc}"
        )

    # Run simulation until termination
    for t in range(1, 501):

        action = 0   # NOOP → only velocity drives drift out of viewport

        state, r, done, truncated, info = env.step(action)
        total_reward += r

        print_step(t, action, state, r, total_reward)

        if done:
            print("\nEpisode terminated at step", info["step_count"])
            print("Reason:", info["terminated_reason"])
            break

    print("Total reward:", total_reward)
    print("Final state:", state)


Out-of-Viewport test initial state: [15.5        10.          5.          0.          1.57079633  0.
  0.          0.        ]
step=001 | action=0 | X=15.600 Y= 9.996 | vx= 5.000 vy=-0.196 | r= -0.010 | R=  -0.010 | L=0 R=0
step=002 | action=0 | X=15.700 Y= 9.988 | vx= 5.000 vy=-0.392 | r= -0.011 | R=  -0.022 | L=0 R=0
step=003 | action=0 | X=15.800 Y= 9.976 | vx= 5.000 vy=-0.589 | r= -0.012 | R=  -0.033 | L=0 R=0
step=004 | action=0 | X=15.900 Y= 9.961 | vx= 5.000 vy=-0.785 | r= -0.013 | R=  -0.046 | L=0 R=0
step=005 | action=0 | X=16.000 Y= 9.941 | vx= 5.000 vy=-0.981 | r= -0.013 | R=  -0.060 | L=0 R=0
step=006 | action=0 | X=16.100 Y= 9.918 | vx= 5.000 vy=-1.177 | r= -0.014 | R=  -0.074 | L=0 R=0

Episode terminated at step 6
Reason: out_of_view
Total reward: -0.0736710994400987
Final state: [16.1         9.917596    5.         -1.1772      1.57079633  0.
  0.          0.        ]
